In [1]:
pip install pycryptodomex

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import sys
import hashlib
import threading
# importing tkinter library for generating graphical user interface
import tkinter as tkint
from pathlib import Path
from tkinter import *
from tkinter import filedialog, messagebox
from Cryptodome.Cipher import AES

In [3]:
class encryption_tool:
    def __init__(self, userfile, userkey, usersalt):
        # getting path to input file
        self.userfile = userfile
        self.inputfile_size = os.path.getsize(self.userfile)
        self.chunksize = 1024
        self.totalchunks = self.inputfile_size // self.chunksize+1
        
        # key and salt conversion into bytes
        self.userkey = bytes(userkey,"utf-8")
        self.usersalt = bytes(userkey[::-1],"utf-8")
        
        # getting file extension
        self.file_ext = self.userfile.split(".")[-1]
        
        # hash type - patented cryptography hash function
        self.hashtype = "SHA256"
        
        # file name during encryption
        self.encrypt_outputfile = (".".join(self.userfile.split(".")[:-1])
                                   + "."
                                   + self.file_ext
                                   + ".encr"
                                  )
        
        # file name during decryption
        self.decrypt_outputfile = self.userfile[:-5].split(".")
        self.decrypt_outputfile = (".".join(self.decrypt_outputfile[:-1])
                                  + "_decrypt."
                                  + self.decrypt_outputfile[-1])
        
        # dictionary to save hash value of key and salt
        self.hashedkey_salt = dict()
        
        # hashing key and salt into hash of 16 bits
        self.hashkey_salt()
        
    # reading file peace by peace 
    def read_chunks(self, fileobject, size_of_chunk=1024):
        while True:
            data = fileobject.read(size_of_chunk)
            if not data:
                break
            yield data
            
    def encrypt(self):
        # creating cipher text
        object_of_cipher = AES.new(self.hashedkey_salt["key"], AES.MODE_CFB, self.hashedkey_salt["salt"])
        
        self.abort() # if output file is already existing then remove that file first
        
        file_of_input = open(self.userfile,"rb")
        file_of_output = open(self.encrypt_outputfile,"ab")
        count_chunks = 0
        
        for chunk in self.read_chunks(file_of_input, self.chunksize):
            encrypt_content = object_of_cipher.encrypt(chunk)
            file_of_output.write(encrypt_content)
            count_chunks += 1
            yield count_chunks / self.totalchunks*100
            
        file_of_input.close()
        file_of_output.close()
        
        del object_of_cipher # cleaning cipher object 
    
    def decrypt(self):
        
        object_of_cipher = AES.new(self.hashedkey_salt["key"], AES.MODE_CFB, self.hashedkey_salt["salt"])
        
        self.abort()
        
        file_of_input = open(self.userfile, "rb")
        file_of_output = open(self.decrypt_outputfile, "xb")
        count_chunks = 0
        
        for chunk in self.read_chunks(file_of_input):
            decrypt_content = object_of_cipher.decrypt(chunk)
            file_of_output.write(decrypt_content)
            count_chunks += 1
            yield count_chunks / self.totalchunks*100
            
        file_of_input.close()
        file_of_output.close()
        
        del object_of_cipher
    
    def abort(self):
        if os.path.isfile(self.encrypt_outputfile):
            os.remove(self.encrypt_outputfile)
        if os.path.isfile(self.decrypt_outputfile):
            os.remove(self.decrypt_outputfile)

            
    def hashkey_salt(self):
        # conversion of key into hash 
        
        hash_object = hashlib.new(self.hashtype)
        hash_object.update(self.userkey)
        
        # converting the output key hash to 32 bytes(256 bits)
        
        self.hashedkey_salt["key"] = bytes(hash_object.hexdigest()[:32], "utf-8")
        
        # cleaning hash object generated
        del hash_object
        
        # conversion of salt into hash 
        
        hash_object = hashlib.new(self.hashtype)
        hash_object.update(self.usersalt)
        
        # converting the output salt value to 16 bytes (128 bits)
        self.hashedkey_salt["salt"] = bytes(hash_object.hexdigest()[:16], "utf-8")
        
        # cleaning hash object generated
        del hash_object
        
        
        
# graphical user-interface formation

class GUIWindow:
    
    # configuring root directory path related to this present program file
    FOLDER_G = ""
    if getattr(sys, "frozen", False):
        FOLDER_G = os.path.dirname(sys.executable)
    else:
        FOLDER_G = os.path.dirname(os.getcwd())
        
    def __init__(self, root):
        self.root = root
        self.cipher = None
        self.file_url = tkint.StringVar()
        self.secret_key = tkint.StringVar()
        self.secret_keycheck = tkint.StringVar()
        self.salt = tkint.StringVar()
        self.status = tkint.StringVar()
        self.status.set("---")
        
        self._shouldcancel = False
        
        root.title("Crypto Encryption")
        root.configure(bg="#eeeeee")
        
        try:
            img_of_icon = tkint.Image(
                            "photo", file= self.FOLDER_G + "./files/encrypc.ico")
            root.call("vm","iconphoto", root._w, img_of_icon)
        except Exception:
            pass
        
        self.menubar = tkint.Menu(root, bg="#eeeeee", relief=tkint.FLAT)
        self.menubar.add_command(label="Help", command=self.callback_show_help)
        self.menubar.add_command(label="About", command=self.about)
        
        root.configure(menu=self.menubar)
        
        self.file_entrylabel = tkint.Label(root, text="Click SELECT FILE or Enter file path",
                                          bg="#eeeeee",
                                          anchor = tkint.W)
        self.file_entrylabel.grid(padx=12,
                                 pady=(8,0),
                                 ipadx=0,
                                 ipady=1,
                                 row=0,
                                 column=0,
                                 columnspan=4,
                                 sticky=tkint.W + tkint.E + tkint.N + tkint.S)
        self.entry_of_file = tkint.Entry(root,
                                        textvariable= self.file_url,
                                        bg="#fff",
                                        exportselection=0,
                                        relief=tkint.FLAT)
        self.entry_of_file.grid(padx=15,
                               pady=6,
                               ipadx=8,
                               ipady=8,
                               row=1,
                               column=0,
                               columnspan=4,
                               sticky= tkint.W + tkint.E + tkint.N + tkint.S)
        self.select_button = tkint.Button(root,
                                         text="SELECT FILE",
                                         command=self.select_file_callback,
                                         width=42,
                                         bg="#3498db",
                                         fg="#ffffff",
                                         bd=2,
                                         relief=tkint.FLAT)
        self.select_button.grid(padx=15,
                               pady=8,
                               ipadx=24,
                               ipady=6,
                               row=2,
                               column=0,
                               columnspan=4,
                               sticky=tkint.W + tkint.E + tkint.N + tkint.S)
        self.key_entrylabel1 = tkint.Label(root,
                                            text="ENTER KEY (Remember key fuurther used in decryption)",
                                            bg="#eeeeee",
                                            anchor=tkint.W)
        self.key_entrylabel1.grid(padx=12,
                                 pady=(8,0),
                                 ipadx=0,
                                 ipady=1,
                                 row=3,
                                 column=0,
                                 columnspan=4,
                                 sticky= tkint.W + tkint.E + tkint.N + tkint.S)
        self.key_entry_1 = tkint.Entry(root,
                                      textvariable=self.secret_key,
                                      bg= "#fff",
                                      exportselection=0,
                                      relief = tkint.FLAT)
        self.key_entry_1.grid(padx=15,
                             pady=6,
                             ipadx=8,
                             ipady=8,
                             row=4,
                             column=0,
                             columnspan=4,
                             sticky=tkint.W + tkint.E + tkint.N + tkint.S)
        self.key_entrylabel2 = tkint.Label(root,
                                          text="Re-enter key",
                                          bg="#eeeeee",
                                          anchor=tkint.W)
        self.key_entrylabel2.grid(padx=12,
                                 pady=(8,0),
                                 ipadx=0,
                                 ipady=1,
                                 row=5,
                                 column=0,
                                 columnspan=4,
                                 sticky= tkint.W + tkint.E + tkint.N + tkint.S)
        self.key_entry_2 = tkint.Entry(root,
                                      textvariable= self.secret_keycheck,
                                      bg ="#fff",
                                      exportselection=0,
                                      relief= tkint.FLAT)
        self.key_entry_2.grid(padx=15,
                             pady=6,
                             ipadx=8,
                             ipady=8,
                             row=6,
                             column=0,
                             columnspan=4,
                             sticky = tkint.W + tkint.E + tkint.N + tkint.S)
        self.encrypt_button = tkint.Button(
            root,
            text="ENCRYPT",
            command=self.encrypt_check_callback,
            bg="#27ae58",
            fg="#ffffff",
            bd=2,
            relief=tkint.FLAT)
        self.encrypt_button.grid(
            padx=15,
            pady=8,
            ipadx=24,
            ipady=6,
            row=7,
            column=0,
            columnspan=2,
            sticky=tkint.W + tkint.E + tkint.N + tkint.S)
        self.decrypt_button = tkint.Button(
            root,
            text="DECRYPT",
            command=self.decrypt_check_callback,
            bg="#27ae58",
            fg="#ffffff",
            bd=2,
            relief=tkint.FLAT)
        self.decrypt_button.grid(
            padx=15,
            pady=8,
            ipadx=24,
            ipady=6,
            row=7,
            column=2,
            columnspan=2,
            sticky=tkint.W + tkint.E + tkint.N + tkint.S)
        self.reset_button = tkint.Button(
            root,
            text="CLEAR",
            command=self.callback_reset,
            bg="#717d7e",
            fg="#ffffff",
            bd=2,
            relief=tkint.FLAT
        )
        self.reset_button.grid(
            padx=15,
            pady=8,
            ipadx=24,
            ipady=6,
            row=8,
            column=0,
            columnspan=2,
            sticky=tkint.W + tkint.E + tkint.N + tkint.S
        )
        self.stop_button = tkint.Button(
            root,
            text="STOP",
            command=self.callback_cancel,
            bg="#aaaaaa",
            fg="#ffffff",
            bd=2,
            state="disabled",
            relief=tkint.FLAT
        )
        self.stop_button.grid(
            padx=15,
            pady=8,
            ipadx=24,
            ipady=6,
            row=8,
            column=2,
            columnspan=2,
            sticky=tkint.W + tkint.E + tkint.N + tkint.S
        )
        self.label_of_status = tkint.Label(
            root,
            textvariable=self.status,
            bg="#eeeeee",
            anchor=tkint.W,
            justify=tkint.LEFT,
            relief=tkint.FLAT,
            wraplength=350
        )
        self.label_of_status.grid(
            padx=12,
            pady=(0, 12),
            ipadx=0,
            ipady=1,
            row=9,
            column=0,
            columnspan=4,
            sticky=tkint.W + tkint.E + tkint.N + tkint.S
        )

        tkint.Grid.columnconfigure(root, 0, weight=1)
        tkint.Grid.columnconfigure(root, 1, weight=1)
        tkint.Grid.columnconfigure(root, 2, weight=1)
        tkint.Grid.columnconfigure(root, 3, weight=1)
    
    def select_file_callback(self):
        try:
            name = filedialog.askopenfile()
            self.file_url.set(name.name)
        except Exception as e:
            self.label_of_status.update()
            
    def control_freeze(self):
        self.entry_of_file.configure(state="disabled")
        self.key_entry_1.configure(state="disabled")
        self.key_entry_2.configure(state="disabled")
        self.select_button.configure(state="disabled", bg="#aaaaaa")
        self.encrypt_button.configure(state="disabled", bg="#aaaaaa")
        self.decrypt_button.configure(state="disabled", bg="#aaaaaa")
        self.reset_button.configure(state="disabled", bg="#aaaaaa")
        self.stop_button.configure(state="normal", bg="#e74c3c")
        self.label_of_status.update()

    def control_unfreeze(self):
        self.entry_of_file.configure(state="normal")
        self.key_entry_1.configure(state="normal")
        self.key_entry_2.configure(state="normal")
        self.select_button.configure(state="normal", bg="#3498db")
        self.encrypt_button.configure(state="normal", bg="#27ae58")
        self.decrypt_button.configure(state="normal", bg="#27ae58")
        self.reset_button.configure(state="normal", bg="#717d7e")
        self.stop_button.configure(state="disabled", bg="#aaaaaa")
        self.label_of_status.update()
        
    def encrypt_check_callback(self):

        newPath = Path(self.file_url.get())
        if newPath.is_file():
            pass
        else:
            messagebox.showinfo("Crypto Encryption", "please enter a valid file URL !")
            return

        if len(self.secret_key.get()) == 0:
            messagebox.showinfo("Crypto Encryption", "Enter a valid secret key !")
            return
        elif self.secret_key.get() != self.secret_keycheck.get():
            messagebox.showinfo("Crypto Encryption", "No match with password !")
            return

        self.callback_to_encrypt()
        
    def decrypt_check_callback(self):

        newPath = Path(self.file_url.get())
        if newPath.is_file():
            pass
        else:
            messagebox.showinfo("Crypto Encryption", "enter a valid File URL please!")
            return

        if self.file_url.get()[-4:] != "encr":
            messagebox.showinfo(
                "EncrypC",
                """file provided is not an encrypted file,
So please enter a valid Encrypted File to do Decryption.""",
            )
            return

        if len(self.secret_key.get()) == 0:
            messagebox.showinfo("Crypto Encryption", "Enter your secret key please")
            return
        elif self.secret_key.get() != self.secret_keycheck.get():
            messagebox.showinfo("Crypto Encryption", "No match with password !")
            return

        self.callback_to_decrypt()
     
    def callback_to_encrypt(self):
        t_1 = threading.Thread(target=self.execute_encrypt)
        t_1.start()

    def execute_encrypt(self):
        self.control_freeze()

        try:
            self.cipher = encryption_tool(
                self.file_url.get(), self.secret_key.get(), self.salt.get()
            )
            for percentage in self.cipher.encrypt():
                if self._shouldcancel:
                    break
                percentage = "{0:.2f}%".format(percentage)
                self.status.set(percentage)
                self.label_of_status.update()

            if self._shouldcancel:
                self.cipher.abort()
                self._status.set("Cancellation completed !")
                messagebox.showinfo("Crypto Encryption", "Cancellation completed !")
                self.cipher = None
                self._shouldcancel = False
                self.control_unfreeze()
                return

            self.cipher = None
            self._shouldcancel = False
            self.status.set("File Encryption done !")
            messagebox.showinfo("Crypto Encryption", "File Encryption done !")
        except Exception as e:

            self.status.set(e)

        self.control_unfreeze()
    
    def callback_to_decrypt(self):
        t_2 = threading.Thread(target=self.execute_decrypt)
        t_2.start()

    def execute_decrypt(self):
        self.control_freeze()

        try:
            self.cipher = encryption_tool(
                self.file_url.get(), self.secret_key.get(), self.salt.get()
            )
            for percentage in self.cipher.decrypt():
                if self._shouldcancel:
                    break
                percentage = "{0:.2f}%".format(percentage)
                self.status.set(percentage)
                self.label_of_status.update()

            if self._shouldcancel:
                self.cipher.abort()
                self.status.set("Cancellation completed !")
                messagebox.showinfo("Crypto Encryption", "Cancellation completed !")
                self.cipher = None
                self._shouldcancel = False
                self.control_unfreeze()
                return

            self.cipher = None
            self._shouldcancel = False
            self.status.set("File Decryption done !")
            messagebox.showinfo("Crypto Encryption", "File Decryption done !")
        except Exception as e:

            self.status.set(e)

        self.control_unfreeze()

    def callback_reset(self):
        self.cipher = None
        self.file_url.set("")
        self.secret_key.set("")
        self.salt.set("")
        self.status.set("---")

    def callback_cancel(self):
        self._shouldcancel = True

    def callback_show_help(self):
        messagebox.showinfo(
            "Tutorial",
            """1. Open the Application and Click SELECT FILE Button to select your file e.g. "mydoc.pdf" (OR You can add path manually).
2. Enter your Key (This should be alphanumeric letters). Remember this so you can Decrypt the file later. (Else you'll lose your file permanently)
3. Click ENCRYPT Button to encrypt the file. A new encrypted file with ".encr" extention e.g. "mydoc.pdf.encr" will be created in the same directory where the "mydoc.pdf" is.
4. When you want to Decrypt a file you, will select the file with the ".encr" extention and Enter your Key which you chose at the time of Encryption. Click DECRYPT Button to decrypt. The decrypted file will be of the same name as before with the suffix "decrypted" for e.g. "mydoc_decrypted.pdf".
5. Click CLEAR Button to reset the input fields and status bar.""",
        )

    def about(self):
        messagebox.showinfo(
            "Cryptography encryption"
        )

if __name__ == "__main__":
    _root = tkint.Tk()
    GUI_WINDOW = GUIWindow(_root)
    bundle_of_dir = getattr(sys, "_MEIPASS", os.path.abspath(os.path.dirname(os.getcwd())))
    path_to_ico = os.path.abspath(os.path.join(bundle_of_dir, "encrypc.ico"))
    _root.iconbitmap(path_to_ico)
    _root.resizable(height=False, width=False)
    _root.mainloop()
    
        
            